<a href="https://colab.research.google.com/github/radicalcoder/ML-Project/blob/master/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import os
import random
import matplotlib
import matplotlib.pyplot as plt
import skimage
import skimage.data
import skimage.transform
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import itertools
from keras import models
from keras.preprocessing import image
from PIL import Image
from keras.models import Sequential
import keras
import PIL
from skimage import io
from skimage import color
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout



# To read the data directory from repository on github.
%matplotlib inline
!git clone -l -s https://github.com/radicalcoder/ML-Project.git data
%cd data
!ls

Cloning into 'data'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 7406 (delta 34), reused 0 (delta 0), pack-reused 7349
Receiving objects: 100% (7406/7406), 248.13 MiB | 34.28 MiB/s, done.
Resolving deltas: 100% (37/37), done.
Checking out files: 100% (7225/7225), done.
/content/data/data
BelgiumTSC_Testing   MLP_final_initialdraft.ipynb  README.md
BelgiumTSC_Training  MLP.ipynb			   traffic.py


In [0]:
def data_loader(data_dir):
    """
    The data set is read into two lists, as follows:
    list labels[] -> a list of numbers, where each number represents a unique image label.
    list images[] -> a list of arrays, where each array represents a single image.
    
    The data set directory contains multiple subdirectories. We read all these 
    subdirectories into a list subdirs[]. Since the subfolders are read in a random order, the list
    is also in a random order. We then sort this array using the sort() function.
    """
    
    subdirs = []
    for i in os.listdir(data_dir):
      if os.path.isdir(data_dir + i):   # There may be .txt files in the data_dir hence we need to check
        subdirs.append(i)

    subdirs.sort()
    
    """
    Data is now collected into two lists - labels & images - while looping through the 
    list subdirs[].    
    """

    labels = []
    images = []
    for i in subdirs:
        label_dir = data_dir + i
        #print(label_dir)
        f_names=[]
        for f in os.listdir(label_dir):
          if f.endswith(".ppm"):   # only files that end in .ppm are image files
            f_names.append(label_dir + '/' + f)
        # print(f_names)
        """
        Now for each label, append all its images into the list images[] and 
        append its label number in the list labels[]. 
        """
        
        for f in f_names:
            images.append(skimage.data.imread(f))   #use parameter as_grey??
            labels.append(int(i))
    return labels, images


# Loading the training and testing dataset using the function data_loader().
training_data_dir = "BelgiumTSC_Training/Training/"
testing_data_dir = "BelgiumTSC_Testing/Testing/"

labels, images = data_loader(training_data_dir)
#print(labels)

In [44]:
print("No of unique Labels: ", len(set(labels)), "\nTotal number of Images: ", len(images)) 

No of unique Labels:  62 
Total number of Images:  4575


In [0]:
def display_images_and_labels(images, labels):
    #Display the first image of each label.
    unique_labels = set(labels)
    plt.figure(figsize=(15, 15))
    i = 1
    for label in unique_labels:
        # Pick the first image for each label.
        image = images[labels.index(label)]
        plt.subplot(8, 8, i)  # A grid of 8 rows x 8 columns
        plt.axis('off')
        plt.title("Label {0} ({1})".format(label, labels.count(label)))
        i += 1
        _ = plt.imshow(image,cmap="gray")
    plt.show()


In [46]:
reshape_images=[]
grayscale_images=[]

for i in images:
  reshape_images.append(skimage.transform.resize(i,(32,32), mode ='constant'))

reshape_images= np.asarray(reshape_images)
#reshape_images = reshape_images.astype('float32')
print(reshape_images.shape)

for i in reshape_images:
  grayscale_images.append(color.rgb2gray(i))
  
grayscale_images= np.asarray(grayscale_images)
#grayscale_images = reshape_images.astype('float32')

print("gray")
print(grayscale_images[0].shape)

image_expanded = grayscale_images[:, :, :, np.newaxis]
grayscale_images=image_expanded

print(grayscale_images[0].shape)

  

#display_images_and_labels(images,labels)
#display_images_and_labels(grayscale_images, labels)

print("Grayscale images")
print(len(grayscale_images))
for i in grayscale_images[:10]:
  print("shape: {0}, min: {1}, max: {2}".format(i.shape, i.min(), i.max()))

print("Reshaped images")

for i in reshape_images[:10]:
  print("shape: {0}, min: {1}, max: {2}".format(i.shape, i.min(), i.max()))

(4575, 32, 32, 3)
gray
(32, 32)
(32, 32, 1)
Grayscale images
4575
shape: (32, 32, 1), min: 0.011294590226715995, max: 0.8266693114276962
shape: (32, 32, 1), min: 0.0721100235523897, max: 0.9055836569393383
shape: (32, 32, 1), min: 0.07370136335784315, max: 0.8950602941176472
shape: (32, 32, 1), min: 0.09544324295343169, max: 0.996078431372549
shape: (32, 32, 1), min: 0.06889145009957118, max: 0.9915034355851715
shape: (32, 32, 1), min: 0.052944715073529366, max: 0.9952450980392157
shape: (32, 32, 1), min: 0.04544354166666667, max: 0.6664398580154718
shape: (32, 32, 1), min: 0.12471631127450947, max: 0.9959773720894608
shape: (32, 32, 1), min: 0.054898770105698505, max: 0.9739451960784319
shape: (32, 32, 1), min: 0.024818349705116323, max: 0.6964905469707414
Reshaped images
shape: (32, 32, 3), min: 0.005943627450980504, max: 0.8743221507352942
shape: (32, 32, 3), min: 0.06296913296568632, max: 0.9191425398284312
shape: (32, 32, 3), min: 0.060355392156862836, max: 0.9028492647058821
shap

In [47]:
print(labels)

from keras.utils import to_categorical
#one-hot encode target column
labels_tr = to_categorical(labels)
print(labels_tr.shape)
print("\n")
print(labels_tr)
print(labels[500])
print(labels_tr[500])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 

In [0]:


from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

model_gen = Sequential()
model_gen.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=grayscale_images[0].shape))
#model_gen.add(Conv2D(32, (3, 3), activation='relu'))
model_gen.add(MaxPooling2D(pool_size=(2, 2)))
model_gen.add(Dropout(0.25))
 
model_gen.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
#model_gen.add(Conv2D(64, (3, 3), activation='relu'))
model_gen.add(MaxPooling2D(pool_size=(2, 2)))
model_gen.add(Dropout(0.25))
 
model_gen.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
#model_gen.add(Conv2D(64, (3, 3), activation='relu'))
model_gen.add(MaxPooling2D(pool_size=(2, 2)))
model_gen.add(Dropout(0.25))
 
model_gen.add(Flatten())
model_gen.add(Dense(512, activation='relu')) # we can drop 
model_gen.add(Dropout(0.5))                  # this layers
model_gen.add(Dense(512, activation='relu'))
model_gen.add(Dropout(0.5))
model_gen.add(Dense(62, activation='softmax'))

In [0]:
model_gen.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [50]:
from keras.preprocessing.image import ImageDataGenerator

# we can use image augmentation
# basically it needs to redifine for normal actual scores like 0.9 of accuracy and more
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=False,
    rotation_range=12,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False)

datagen.fit(grayscale_images)


%time
epochs=50
batch_size = 20
history_generator = model_gen.fit_generator(datagen.flow(grayscale_images, labels_tr, batch_size=batch_size),
                    epochs=epochs, steps_per_epoch=500 , verbose=1,
                    validation_data=(grayscale_images, labels_tr))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs
Epoch 1/50
500/500 [==============================] - 5s 11ms/step - loss: 3.1622 - acc: 0.2145 - val_loss: 2.2365 - val_acc: 0.4114
Epoch 2/50
500/500 [==============================] - 5s 10ms/step - loss: 2.0982 - acc: 0.4530 - val_loss: 1.5826 - val_acc: 0.5591
Epoch 3/50
500/500 [==============================] - 5s 10ms/step - loss: 1.6452 - acc: 0.5576 - val_loss: 1.1884 - val_acc: 0.6647
Epoch 4/50
500/500 [==============================] - 5s 10ms/step - loss: 1.4389 - acc: 0.6090 - val_loss: 1.0717 - val_acc: 0.7047
Epoch 5/50
500/500 [==============================] - 5s 10ms/step - loss: 1.3004 - acc: 0.6430 - val_loss: 1.4420 - val_acc: 0.6203
Epoch 6/50
500/500 [==============================] - 5s 10ms/step - loss: 1.2259 - acc: 0.6641 - val_loss: 1.0037 - val_acc: 0.7242
Epoch 7/50
500/500 [==============================] - 5s 10ms/step - loss: 1.1933 - acc: 0.6705 - val_loss: 1.0820 - val_acc: 0.7014
Epoch

KeyboardInterrupt: ignored